# Terraform configuration

* [Terraform Settings](https://www.terraform.io/docs/language/settings/index.html)

> The special terraform configuration block type is used to configure some behaviors of Terraform itself, such as requiring a minimum Terraform version to apply your configuration.

1. Terraform runtime version
2. Backend
3. Resource providers and their versions 

In [83]:
cat versions.tf

#--------------------------------------------------------------------------------
# [terraform configuration block]
# https://www.terraform.io/docs/language/settings/index.html#specifying-a-required-terraform-version
#--------------------------------------------------------------------------------
terraform {
  #--------------------------------------------------------------------------------
  # required_version
  # https://www.terraform.io/docs/language/expressions/version-constraints.html
  # Version Constraints
  # https://www.terraform.io/docs/language/expressions/version-constraints.html
  #--------------------------------------------------------------------------------
  required_version = "~> 1.0.0"

  #--------------------------------------------------------------------------------
  # workspace backend
  # https://www.terraform.io/docs/language/state/workspaces.html
  # The persistent data stored in the backend belongs to a workspace.
  #---------------------------------------

---
# GCP Project

* [Creating a project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#gcloud)
```gcloud projects create PROJECT_ID```


Make sure to link the project to the billing account. If not linked, the error is caused.
```
AccessDeniedException: 403 The project to be billed is associated with an absent billing account.
```

* [gcloud beta billing ](https://cloud.google.com/sdk/gcloud/reference/beta/billing)

> gcloud beta billing projects link ${PROJECT_ID} --billing-account 0X0X0X-0X0X0X-0X0X0X


```
$ gcloud beta billing projects link ${PROJECT_ID} --billing-account 0X0X0X-0X0X0X-0X0X0X
billingAccountName: billingAccounts/0X0X0X-0X0X0X-0X0X0X
billingEnabled: true
name: projects/terraform-20211001125127/billingInfo
projectId: terraform-20211001125127
```

In [31]:
PROJECT_ID="terraform-$(date +%Y%m%d%H%M%S)"
echo $PROJECT_ID

terraform-20211001125127


In [43]:
ENVIRONMENT="dev"

In [32]:
gcloud projects create $PROJECT_ID
gcloud config set core/project $PROJECT_ID

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/terraform-20211001125127].
Waiting for [operations/cp.7259930078076563699] to finish...done.              
Enabling service [cloudapis.googleapis.com] on project [terraform-20211001125127]...
Operation "operations/acf.p2-25902825767-54e6a0fb-b942-4baf-82d1-872cd533ee5d" finished successfully.
Updated property [core/project].


## Enable APIs

To avoid errors.

```
Error: Error when reading or editing Project : googleapi: Error 403: Cloud Resource Manager API has not been used in project before or it is disabled.
```

In [101]:
gcloud services enable cloudresourcemanager.googleapis.com

# Service Account for Terraform

In [33]:
SERVICE_ACCOUNT_ID="terraform"
SERVICE_DESCRIPTION=f"Terraform service account for ${PROJECT_ID}"
SERVICE_NAME="Terraform" 
SERVICE_ROLE="roles/editor"

In [34]:
gcloud iam service-accounts create $SERVICE_ACCOUNT_ID \
    --description="$SERVICE_DESCRIPTION" \
    --display-name=$SERVICE_NAME \
    --project=$PROJECT_ID

Created service account [terraform].


In [35]:
gcloud iam service-accounts describe $SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com

description: fTerraform service account for terraform-20211001125127
displayName: Terraform
email: terraform@terraform-20211001125127.iam.gserviceaccount.com
etag: MDEwMjE5MjA=
name: projects/terraform-20211001125127/serviceAccounts/terraform@terraform-20211001125127.iam.gserviceaccount.com
oauth2ClientId: '107583246644728722345'
projectId: terraform-20211001125127
uniqueId: '107583246644728722345'


In [36]:
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:$SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com" \
    --role=$SERVICE_ROLE > /dev/null 2>&1

## Allow the local user to impersonate the service account

In [90]:
USER_EMAIL=$(gcloud config get-value core/account)
gcloud iam service-accounts add-iam-policy-binding \
    $SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com \
    --member="user:$USER_EMAIL" \
    --role="roles/iam.serviceAccountTokenCreator" > /dev/null 2>&1

# Cloud Storage for terraform backend

In [64]:
BUCKET_NAME="${PROJECT_ID}-${ENVIRONMENT}-terraform-state"
gsutil mb -p "${PROJECT_ID}" "gs://${BUCKET_NAME}/"

Creating gs://terraform-20211001125127-dev-terraform-state/...


In [70]:
gsutil ls -p ${PROJECT_ID} | grep ${BUCKET_NAME}

gs://terraform-20211001125127-dev-terraform-state/


---

# Terraform Backend

* [Backend Configuration](https://www.terraform.io/docs/language/settings/backends/configuration.html)

Resource states are stored in the backend. Whenever a configuration's backend changes, you must run terraform init again to validate and configure the backend before you can perform any plans, applies, or state operations.


##  Workspace

* [Workspaces](https://www.terraform.io/docs/language/state/workspaces.html)

> The persistent data stored in the backend belongs to a workspace. Initially the backend has only one workspace, called "default".

### Commands for workspace

* [Terfaform command workspace](https://www.terraform.io/docs/cli/commands/workspace/index.html)

> ```terraform workspace <subcommand> [options] [args]```
    
To create a workspace and switch to it:
```terraform workspace new [OPTIONS] NAME [DIR]```
    
To siwtch to a workspace:
```terraform workspace select NAME [DIR]```
    

## Terraform Variables

In [71]:
export TF_VAR_PROJECT_ID=${PROJECT_ID}
export TF_VAR_GCP_TERRAFORM_SERVICE_ACCOUNT_ID="${SERVICE_ACCOUNT_ID}@${PROJECT_ID}.iam.gserviceaccount.com"
export TF_VAR_BACKEND_BUCKET="${BUCKET_NAME}"

echo ${TF_VAR_PROJECT_ID}
echo ${TF_VAR_GCP_TERRAFORM_SERVICE_ACCOUNT_ID}
echo ${TF_VAR_BACKEND_BUCKET}

terraform-20211001125127
terraform@terraform-20211001125127.iam.gserviceaccount.com
terraform-20211001125127-dev-terraform-state


## Initialize Terraform

## Dynamically set the backend configuration

To avoid hard-coding the terraform backend configurations, use the ```backend-config``` command line option.

* [Using variables in terraform backend config block #13022](https://github.com/hashicorp/terraform/issues/13022)

```
terraform {
  backend "azurerm" {
    resource_group_name  = "rg-demo-terraform-simple-storage-container"
    storage_account_name = "<THIS WILL BE REPLACED VIA -backend-config>"  # <---
    container_name       = "tfstate"
    key                  = "mydemo.tfstate"

  }
}

$ terraform init -reconfigure -backend-config="storage_account_name=anotherstorage0123"
```

In [72]:
terraform init -reconfigure -backend-config="bucket=$TF_VAR_BACKEND_BUCKET"


Initializing the backend...

Successfully configured the backend "gcs"! Terraform will automatically
use this backend unless the backend configuration changes.

Initializing provider plugins...
- Finding hashicorp/google versions matching "3.86.0"...
- Installing hashicorp/google v3.86.0...
- Installed hashicorp/google v3.86.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other


In [73]:
terraform workspace new dev

Created and switched to workspace "dev"!

You're now on a new, empty workspace. Workspaces isolate their state,
so if you run "terraform plan" Terraform will not see any existing state
for this configuration.


In [75]:
terraform workspace new uat

Created and switched to workspace "uat"!

You're now on a new, empty workspace. Workspaces isolate their state,
so if you run "terraform plan" Terraform will not see any existing state
for this configuration.


In [81]:
terraform workspace select dev

Switched to workspace "dev".


## Backend state files created
<img src="image/backend_workspace_state_files.png" align="left" width="600"/>

---
# GCP resource provision

## GCP Provider

In [102]:
cat gcp.tf

data "google_project" "current" {
  #--------------------------------------------------------------------------------
  # [Datarouce google_project]
  # https://registry.terraform.io/providers/hashicorp/google/latest/docs/data-sources/project
  # [Resource google_project]
  # https://registry.terraform.io/providers/hashicorp/google/latest/docs/resources/google_project
  #--------------------------------------------------------------------------------
  project_id = var.PROJECT_ID
}

output "project_number" {
  value = data.google_project.current.number
}

output "project_id" {
  value = data.google_project.current.project_id
}
variable "GCP_TERRAFORM_SERVICE_ACCOUNT_ID" {
  type = string
  description = "Service account to impersonate"
}

provider "google" {
  #--------------------------------------------------------------------------------
  # [Google Provider Configuration Reference]
  # https://registry.terraform.io/providers/hashicorp/google/latest/docs/guides/provider_reference
  

In [97]:
terraform plan

Acquiring state lock. This may take a few moments...

No changes. Your infrastructure matches the configuration.

Terraform has compared your real infrastructure against your configuration and
found no differences, so no changes are needed.


In [99]:
terraform apply --auto-approve


Changes to Outputs:
  + project_id     = "terraform-20211001125127"
  + project_number = "25902825767"

You can apply this plan to save these new output values to the Terraform state,
without changing any real infrastructure.

Apply complete! Resources: 0 added, 0 changed, 0 destroyed.

Outputs:

project_id = "terraform-20211001125127"
project_number = "25902825767"
